The goal of this competition is to predict monthly microbusiness density in a given area.

Files:
External data sources may be used for county features

train.csv
test.csv
sample_submission.csv
census_starter.csv

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context('notebook')

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
census = pd.read_csv('census_starter.csv')

In [4]:
train.shape

(122265, 7)

In [5]:
train.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [6]:
train.dtypes

row_id                    object
cfips                      int64
county                    object
state                     object
first_day_of_month        object
microbusiness_density    float64
active                     int64
dtype: object

row_id is just cfips and date concatenated

In [7]:
train = train.astype({'cfips':'category', 'county':'category', 
    'state':'category', 'first_day_of_month':'datetime64[ns]'})
train.dtypes

row_id                           object
cfips                          category
county                         category
state                          category
first_day_of_month       datetime64[ns]
microbusiness_density           float64
active                            int64
dtype: object

In [8]:
train.head()

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243


In [9]:
train.isnull().sum()

row_id                   0
cfips                    0
county                   0
state                    0
first_day_of_month       0
microbusiness_density    0
active                   0
dtype: int64

In [10]:
train.nunique()

row_id                   122265
cfips                      3135
county                     1871
state                        51
first_day_of_month           39
microbusiness_density     97122
active                    19193
dtype: int64

Counties have duplicate names. Dropping county column. Also, row_id can be dropped.

In [11]:
train.drop(columns = ['row_id', 'county'], inplace = True)

In [12]:
train['first_day_of_month'].min(), train['first_day_of_month'].max()

(Timestamp('2019-08-01 00:00:00'), Timestamp('2022-10-01 00:00:00'))

No missing months overall

In [13]:
39*3135

122265

No missing months for any county

In [14]:
any(train.duplicated(subset=['cfips', 'first_day_of_month']))

False

In [15]:
train['first_day_of_month'].dt.day.value_counts()

1    122265
Name: first_day_of_month, dtype: int64

In [16]:
train.head()

,cfips,state,first_day_of_month,microbusiness_density,active
0,1001,Alabama,2019-08-01,3.007682,1249
1,1001,Alabama,2019-09-01,2.884870,1198
2,1001,Alabama,2019-10-01,3.055843,1269
3,1001,Alabama,2019-11-01,2.993233,1243
4,1001,Alabama,2019-12-01,2.993233,1243


In [17]:
train.dtypes

cfips                          category
state                          category
first_day_of_month       datetime64[ns]
microbusiness_density           float64
active                            int64
dtype: object

In [18]:
train['month'] = train['first_day_of_month'].dt.month
train['year'] = train['first_day_of_month'].dt.year

In [19]:
train['month'] = train['month'].astype('int')
train['year'] = train['year'].astype('int')

In [20]:
train.head()

,cfips,state,first_day_of_month,microbusiness_density,active,month,year
0,1001,Alabama,2019-08-01,3.007682,1249,8,2019
1,1001,Alabama,2019-09-01,2.884870,1198,9,2019
2,1001,Alabama,2019-10-01,3.055843,1269,10,2019
3,1001,Alabama,2019-11-01,2.993233,1243,11,2019
4,1001,Alabama,2019-12-01,2.993233,1243,12,2019


In [21]:
train.describe()

,microbusiness_density,active,month,year
count,122265.000000,1.222650e+05,122265.000000,122265.000000
mean,3.817671,6.442858e+03,6.692308,2020.692308
std,4.991087,3.304001e+04,3.390452,0.991089
min,0.000000,0.000000e+00,1.000000,2019.000000
25%,1.639344,1.450000e+02,4.000000,2020.000000
50%,2.586543,4.880000e+02,7.000000,2021.000000
75%,4.519231,2.124000e+03,10.000000,2022.000000
max,284.340030,1.167744e+06,12.000000,2022.000000


In [22]:
train.loc[train['microbusiness_density']>100]

,cfips,state,first_day_of_month,microbusiness_density,active,month,year
68658,32510,Nevada,2021-02-01,154.77109,67409,2,2021
68659,32510,Nevada,2021-03-01,155.50581,67729,3,2021
68660,32510,Nevada,2021-04-01,169.16472,73678,4,2021
68661,32510,Nevada,2021-05-01,181.05341,78856,5,2021
68662,32510,Nevada,2021-06-01,184.43082,80327,6,2021
68663,32510,Nevada,2021-07-01,189.30293,82449,7,2021
68664,32510,Nevada,2021-08-01,194.23015,84595,8,2021
68665,32510,Nevada,2021-09-01,202.78964,88323,9,2021
68666,32510,Nevada,2021-10-01,204.76649,89184,10,2021
68667,32510,Nevada,2021-11-01,206.80765,90073,11,2021


In [23]:
test.dtypes

row_id                object
cfips                  int64
first_day_of_month    object
dtype: object

In [24]:
test = test.astype({'cfips':'category', 'first_day_of_month':'datetime64[ns]'})

In [26]:
test.dtypes

row_id                        object
cfips                       category
first_day_of_month    datetime64[ns]
dtype: object

In [27]:
test.nunique()

row_id                25080
cfips                  3135
first_day_of_month        8
dtype: int64

In [28]:
test['first_day_of_month'].min(), test['first_day_of_month'].max()

(Timestamp('2022-11-01 00:00:00'), Timestamp('2023-06-01 00:00:00'))

In [29]:
8*3135

25080

So we need prediction for the next 8 months for all 3135 counties